In [1]:
import numpy as np
import pandas as pd

In [89]:
trade_position = pd.Series(np.random.randint(-3,3, 10))
trade_position

0   -1
1   -2
2    0
3    2
4    1
5    0
6    2
7    2
8    1
9    0
dtype: int32

In [56]:
# 포지션이 바뀐 날 탐색
## df0 : 거래 0인 날(포지션 변동)
df0 = trade_position[trade_position==0].index 
## df1 : 전날 거래가 0이 아닌 날
df1 = trade_position.shift(1)
df1 = df1[df1!=0].index
## bets : 전날 포지션이 0이 아닌데, 현재 0인 날(0 연속 방지)
bets = df0.intersection(df1)
## 전날 포지션과 다음날 포지션 곱함, 음수라면 포지션 변동
df0 = trade_position.iloc[1:] * trade_position.iloc[:-1].values
## 전날 0이 아닌데 현재 0 + 전날과 다음날 포지션 변동일
bets = bets.union(df0[df0<0].index).sort_values()
## 마지막 날이 포함되어 있지 않으면 추가
if trade_position.index[-1] not in bets:
    bets.append(trade_position.index[-1:])

Int64Index([1, 4, 7, 9], dtype='int64')

In [59]:
def getHoldingPeriod(tPos):
    ## DataFrame, 0 생성
    hp, tEntry = pd.DataFrame(columns = ["dT", "w"]), 0.
    ## pDiff : 일자별 변동량, tDiff : 첫날부터 index 일까지의 날짜 수
    pDiff, tDiff = tPos.diff(), (tPos.index - tPos.index[0]) / \
                                 p.timedelta64(1, "D")
    ## 1부터 총 거래일까지
    for i in range(1, tPos.shape[0]):
        ## 일자별 변동량 * 전날 포지션
        ## 포지션 증가(매수 상태에서 매수, 매도 상태에서 매도) or 변화 없음
        if pDiff.iloc[i]*tPos.iloc[i-1] >= 0:
            ## 해당 포지션이 0이 아니라면
            if tPos.iloc[i] != 0:
                tEntry = (tEntry * tPos.iloc[i -1] + tDiff[i] * pDiff.iloc[i]) / tPos.iloc[i]
        ## 포지션 감소
        else:
            ## 포지션 변경(매수 상태에서 매도, 매도 상태에서 매수 )
            if tPos.iloc[i] * tPos.iloc[i-1] < 0:
                hp.loc[tPos.index[i], ["dT", "w"]] = (tDiff[i] - tEntry, abs(tPos.iloc[i-1]))
                ## 진입 시점 초기화
                tEntry = tDiff[i] 
            ## 포지션 변경 X
            else:
                hp.loc[tPos.index[i], ["dT", "w"]] = (tDiff[i] - tEntry, abs(pDiff.iloc[i]))
    ## 포지션 보유시
    if hp["w"].sum() > 0:
        ## 평균 계산
        hp = (hp["dT"] * hp["w"]).sum() / hp["w"].sum()
    ## hp["w"].sum() == 0 : 포지션 보유 x
    else:
        hp = np.nan
    return hp
    

In [90]:
trade_position

0   -1
1   -2
2    0
3    2
4    1
5    0
6    2
7    2
8    1
9    0
dtype: int32

In [91]:
trade_position.diff()

0    NaN
1   -1.0
2    2.0
3    2.0
4   -1.0
5   -1.0
6    2.0
7    0.0
8   -1.0
9   -1.0
dtype: float64

In [92]:
trade_position.shift(1)

0    NaN
1   -1.0
2   -2.0
3    0.0
4    2.0
5    1.0
6    0.0
7    2.0
8    2.0
9    1.0
dtype: float64

In [93]:
trade_position.diff() * trade_position.shift(1)

0    NaN
1    1.0
2   -4.0
3    0.0
4   -2.0
5   -1.0
6    0.0
7    0.0
8   -2.0
9   -1.0
dtype: float64

0   -2.0
1   -3.0
2   -2.0
3    0.0
4   -2.0
5   -3.0
6    0.0
7   -4.0
8    2.0
9    NaN
dtype: float64

In [49]:
bets

Int64Index([1, 4, 7, 9], dtype='int64')